In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np


df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MNIST_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MNIST_test.csv')

df_filt_train = df_train[df_train['label'].isin([6, 7])]
df_filt_test = df_test[df_test['label'].isin([6, 7])]

x_train = df_filt_train.drop('label', axis=1)
y_train = df_filt_train['label']
x_test = df_filt_test.drop('label', axis=1)
y_test = df_filt_test['label']

pca = PCA(svd_solver='full')
pca.fit(x_train)

PCA(svd_solver='full')

In [3]:
len(df_train), len(df_test)

(60000, 10000)

In [4]:
evr = np.cumsum(pca.explained_variance_ratio_)
np.where(evr > 0.84)[0][0] + 1

46

In [5]:
pca = PCA(svd_solver='full', n_components=46)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [6]:
x_train_pca[13][8]

-66.86359746012437

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix


gbc = GradientBoostingClassifier(n_estimators=500, learning_rate=0.8, random_state=11, max_depth=2)
gbc.fit(x_train_pca, y_train)
y_pred = gbc.predict(x_test_pca)

In [8]:
classification_report(y_test, y_pred, output_dict=True)["accuracy"]

0.9934541792547835

In [9]:
confusion_matrix(y_test, y_pred)

array([[ 951,    7],
       [   6, 1022]])

In [10]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


bc = BaggingClassifier(
    DecisionTreeClassifier(class_weight='balanced'),
    max_samples=0.5,
    max_features=0.5,
    bootstrap=False,
  random_state=11)
bc.fit(x_train_pca, y_train)
y_pred_bc = bc.predict(x_test_pca)

In [11]:
classification_report(y_test, y_pred_bc, output_dict=True)["accuracy"]

0.9909365558912386

In [12]:
confusion_matrix(y_test, y_pred_bc)

array([[ 951,    7],
       [  11, 1017]])